## Running Backs

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

First, we pull 2022 offense data and clean it

In [31]:
df, y = Fantasy.read_data('2021')

Offense_2020
Offense_Proj
Offense_Prior_Actuals
Offense_Proj


Split data by positional group

In [45]:
RB_data = football.makeRegularData(df, "RB")

Next, we pull more advanced data for rushing and passing

In [43]:
# Passing advanced
rushing_advanced = football.getAdvancedStats("rushing", "2021")

In [42]:
receiving_advanced = football.getAdvancedStats("receiving", "2021") 

In [46]:
rushing_advanced = rushing_advanced.head(120)

In [47]:
temp_rushing = rushing_advanced[rushing_advanced['Pos'] == 'RB']
temp_receiving = receiving_advanced[receiving_advanced['Pos'] == 'RB']

In [58]:
RB_rushing = football.addAdvanced(RB_data, temp_rushing)
RB_receiving = football.readyToMerge(temp_receiving)
RB1 = football.secondMerge(RB_rushing, RB_receiving)
RB = football.simpleMerge(RB1, y)
RB_final = football.fix_columns(RB)
RB_final = RB_final.loc[:,~RB_final.columns.duplicated()]

In [59]:
RB_final

,Player,Pos,Tm,GP,Att,Rushing_Yds,Rushing_Td,Rushing_1st,Tgt,Rec,...,Att/Br,YBC/R,YAC/R,ADOT,Rec/Br,Drop,Drop%,Int,Rat,points
0,N. Chubb,RB,Cle,12,190,1067,12,55,18,16,...,9.9,0.1,8.6,1.0,5.0,0,0.0,0,109.0,195.3
1,A. Jones,RB,GB,14,201,1104,9,52,63,47,...,14.3,-0.4,7.9,0.9,13.0,3,4.6,0,122.5,177.0
2,J. Taylor,RB,Ind,15,232,1169,11,69,39,36,...,13.3,-1.2,10.2,-0.9,20.0,5,9.8,0,109.2,333.1
3,J. Taylor,RB,Ind,15,232,1169,11,69,39,36,...,13.3,-0.5,2.5,-0.5,,0,0.0,0,79.2,333.1
4,S. Barkley,RB,NYG,2,19,34,0,1,9,6,...,40.5,-0.9,7.3,0.5,10.3,6,10.5,0,92.9,107.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,K. Johnson,RB,Phi,16,52,181,2,9,26,19,...,0,0,0,0,0,0,0,0,0,0.0
183,J. Hawkins,RB,Ten,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0.0
184,L. Perine,RB,NYJ,10,64,232,2,18,15,11,...,0,0,0,0,0,0,0,0,0,0.0
185,A. McFarland Jr.,RB,Pit,11,33,113,0,4,9,6,...,0,0,0,0,0,0,0,0,0,0.0


### CONTRACTS AND OTHER DATA (not yet completed)

We are not using the player contracts yet but here is where we scrape that data

In [38]:
table_contracts = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/offense/all-time/free-agent-2021/limit-100/')

player_contracts = table_contracts[0]
player_contracts

,Rank,Player,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed
0,1,Jay Cutler Quarterback | 2014-2020 (FA: 2021)...,30,7,"$126,700,000","$18,100,000",NaN,"$38,000,000","$54,000,000"
1,2,Cam Newton Quarterback | 2015-2020 (FA: 2021),26,5,"$103,800,000","$20,760,000","$22,500,000","$41,000,000","$60,000,000"
2,3,Andy Dalton Quarterback | 2014-2020 (FA: 2021...,26,6,"$96,000,000","$16,000,000","$12,000,000","$17,000,000","$17,000,000"
3,4,Kirk Cousins Quarterback | 2018-2020 (FA: 2021),29,3,"$84,000,000","$28,000,000","$3,000,000","$84,000,000","$84,000,000"
4,5,Ryan Tannehill Quarterback | 2015-2020 (FA: 2...,26,4,"$77,000,000","$19,250,000","$11,500,000","$21,500,000","$45,000,000"
...,...,...,...,...,...,...,...,...,...
95,96,Paxton Lynch Quarterback | 2016-2020 (FA: 202...,22,4,"$9,476,296","$2,369,074","$5,091,852","$7,884,074","$7,884,074"
96,97,Lee Smith Tight End | 2018-2020 (FA: 2021) O...,30,3,"$9,000,000","$3,000,000",NaN,"$3,000,000","$3,000,000"
97,97,Sammy Watkins Wide Receiver | 2020-2020 (FA: ...,26,1,"$9,000,000","$9,000,000","$1,000,000","$8,000,000","$8,000,000"
98,97,Donte Moncrief Wide Receiver | 2019-2020 (FA:...,25,2,"$9,000,000","$4,500,000","$3,500,000","$3,500,000","$3,500,000"
